# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_data.csv")

# Display sample data
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,53.22,66,0,21.85,AR,1701984712
1,1,namibe,-15.1961,12.1522,73.71,77,98,5.97,AO,1701984904
2,2,whitehorse,60.7161,-135.0538,28.02,79,75,15.01,CA,1701984904
3,3,mangrol,21.1167,70.1167,71.24,57,1,11.48,IN,1701984904
4,4,soc trang,9.6033,105.9800,76.32,85,88,8.46,VN,1701984904


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display




# Create a points plot using hvPlot with size based on Humidity
points = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                    color ='City', alpha=0.6)


# Customize the plot
gv_plot.opts(
    gv.opts.Points(width=800, height=600, size_index=3, global_extent=True),
    gv.opts.WMTS(width=800, height=600, global_extent=True)
)
#Display Map
gv_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Humidity'] < 40) & 
                               (city_data_df['Max Temp'] >= 65)  
                               ]
# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,lau,9.2167,11.2833,78.64,17,8,3.27,NG,1701984905
57,57,nova xavantina,-14.6667,-52.3500,91.29,32,68,2.10,BR,1701984915
90,90,villa de soto,-30.8552,-64.9995,82.40,24,37,16.13,AR,1701984924
115,115,codrington,-38.2667,141.9667,82.80,36,99,19.15,AU,1701984930
116,116,calama,-22.4667,-68.9333,79.29,5,0,28.77,CL,1701984930


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,lau,NG,9.2167,11.2833,17,
57,nova xavantina,BR,-14.6667,-52.3500,32,
90,villa de soto,AR,-30.8552,-64.9995,24,
115,codrington,AU,-38.2667,141.9667,36,
116,calama,CL,-22.4667,-68.9333,5,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {
"categories": "accommodation.hotel",
    "limit": 1,  # Assuming you want just the nearest hotel
    "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lau - nearest hotel: No hotel found
nova xavantina - nearest hotel: Hotel Morada do Sol
villa de soto - nearest hotel: La Eleida
codrington - nearest hotel: No hotel found
calama - nearest hotel: Apart Hotel Geotel Calama
dongola - nearest hotel: Lord
ridgecrest - nearest hotel: Best Western China Lake Inn
kidal - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
niamey - nearest hotel: Redidence Croix du Sud
deer valley - nearest hotel: Days Inn Phoenix
ngaoundere - nearest hotel: LA CONCORDE
recreo - nearest hotel: Hotel Sarmiento
zouerate - nearest hotel: فندق تيرس زمور
dabola - nearest hotel: Hotel Tinkisso, rooms from 200k
vryburg - nearest hotel: International Hotel
coyhaique - nearest hotel: No hotel found
tchintabaraden - nearest hotel: No hotel found
mandera - nearest hotel: Kornesh Hotel and Lodging
rumbek - nearest hotel: Hillview Hotel
alaghsas - nearest hotel: Hôtel Zagado
ipora - nearest hotel: No hotel found
golere - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
12,lau,NG,9.2167,11.2833,17,No hotel found
57,nova xavantina,BR,-14.6667,-52.3500,32,Hotel Morada do Sol
90,villa de soto,AR,-30.8552,-64.9995,24,La Eleida
115,codrington,AU,-38.2667,141.9667,36,No hotel found
116,calama,CL,-22.4667,-68.9333,5,Apart Hotel Geotel Calama
133,dongola,SD,19.1667,30.4833,21,Lord
140,ridgecrest,US,35.6225,-117.6709,28,Best Western China Lake Inn
170,kidal,ML,18.4411,1.4078,13,No hotel found
176,bardai,NG,11.6500,7.2333,20,No hotel found
197,niamey,NE,13.5137,2.1098,12,Redidence Croix du Sud


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display
import geoviews.tile_sources as gts


pointss = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                hover_cols=['City', 'Hotel Name'], c='City', cmap='Category20', alpha=0.6)

# Apply options to the points plot
pointss = pointss.opts(size_index=3, tools=['hover'])

# Overlay points on the GeoViews tile source
gvv_plot = gv.tile_sources.OSM() * pointss

# Customize the overall plot
gvv_plot = gvv_plot.opts(width=800, height=600, global_extent=True)

# Display the map
gvv_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)